In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from textblob import TextBlob
import csv,re,time

In [20]:
def get_platform(text,platform_value):
    blob = TextBlob(text)
    for word,tag in blob.tags:
        #print(word,tag,"\n")
        if tag=="NN":
            if word!="stated":
                platform_value=word
    Months=["January","February","March","April","May","June","July","August","September","October","November","December","january","february","march","april","may","june","july","august","september","october","november","december"]
    for i in blob.noun_phrases:
        if i not in Months:
            platform_value=i
    return platform_value

In [21]:
def get_links_from_page(writer:csv.writer):

    results=driver.find_elements(by=By.CSS_SELECTOR,value='article[class*="m-statement m-statement--is-medium m-statement"]')
    
    for result in results: # for each story div
        
        date,platform,text,name,truth='NA','NA','NA','NA','NA'
       
        try: # try to get the name
            name_box=result.find_element(by=By.CSS_SELECTOR,value='a[class="m-statement__name"]')
            name=name_box.text

        except NoSuchElementException as e:
            print('could not extract name')
            
        try: 
            data_box=result.find_element(by=By.CSS_SELECTOR,value='div[class="m-statement__desc"]')
            fact_date=data_box.text
            #print(fact_date)
            date_object = re.findall(r'(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{1,2}, )?\d{2,4}',fact_date)
            date = ' '.join(map(str, date_object))
            #print(date)
            platform_value="Platform is not mentioned"
            platform=get_platform(data_box.text,platform_value)
            #print(platform)

        except NoSuchElementException as e:
            print('could not extract date')
            
        try: 
            review_text_box=result.find_element(by=By.CSS_SELECTOR,value='div[class="m-statement__quote"]')
            text=review_text_box.text
            #print(text)

        except NoSuchElementException as e:
            print('could not extract text')
            
        try:
            truthfullness_box=result.find_element(by=By.CSS_SELECTOR,value='div[class="m-statement__meter"]')
            truth_data = truthfullness_box.find_element(by=By.CSS_SELECTOR,value='img[class="c-image__thumb"]')
            truth = truth_data.get_attribute('alt')

        except NoSuchElementException as e:
            print('could not extract truthfullness_category')
        
        #write a new row for this story
        writer.writerow([name,date,platform,text,truth])

In [22]:
def scrape(driver,url):
    delay:int = 15   # number of seconds to wait 

    driver.maximize_window()
    driver.get(url)  # visit the page

    #input() # wait until we can click on the cookie overlay

    page_count = 1  # keep track of page count

    # create a new csv writer for the review data
    fw=open('Output.csv','w',encoding="utf-8")  # output file

    writer=csv.writer(fw,lineterminator='\n')   # create a csv writer for this file
    
    writer.writerow(['Name','Date','Platform','Text','Truthfulness Category'])

    while True: # keep going until there are no more pages

        print("Page No: ",page_count)   # print current page count

        page_count+=1   # increment page count

        get_links_from_page(writer)    # extract and write the review data from the current page
        
        try:
            # wait until the next button appears
            next_button = WebDriverWait(driver, delay).until(EC.element_to_be_clickable((By.LINK_TEXT, 'Next'))) 
            
            if (page_count == 5):  #test case for running upto 5 page
                break
        
        except TimeoutException:
            break

        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(delay)
        
    fw.close()    

In [23]:
url = 'https://www.politifact.com/factchecks/list/?category=truth-o-meter&ruling='
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) 

In [24]:
scrape(driver,url)

Page No:  1
Page No:  2
Page No:  3
Page No:  4
